In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
dataset = pd.read_csv('amazonreviews.tsv', delimiter = '\t', quoting = 3)

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 10000):
  review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i]) # Remove all the punctuations
  review = review.lower()
  review = review.split() # Split it to different elements (words)
  ps = PorterStemmer() 
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 0].values

In [5]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
print(y)

['pos' 'pos' 'pos' ... 'neg' 'neg' 'pos']


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [8]:
print(y)

[1 1 1 ... 0 0 1]


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [10]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
print(X_train)

[[-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 ...
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815  5.73622459 ... -0.04950514 -0.06977019
  -0.05130213]
 [-0.07856397 -0.15847815 -0.16069095 ... -0.04950514 -0.06977019
  -0.05130213]]


# Building the Neaural Network

In [13]:
ann = tf.keras.models.Sequential()

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [16]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 50)

Epoch 1/50
250/250 [==============================] - 0s 1ms/step - loss: 0.6378 - accuracy: 0.6675
Epoch 2/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3343 - accuracy: 0.8633
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.2076 - accuracy: 0.9189
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.1383 - accuracy: 0.9464
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0912 - accuracy: 0.9646
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0598 - accuracy: 0.9775
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0415 - accuracy: 0.9841
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0306 - accuracy: 0.9875
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0249 - accuracy: 0.9891
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.0214 - accuracy: 0.9893

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 0]
 [1 1]
 [0 0]
 ...
 [1 1]
 [0 0]
 [1 1]]


In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[847 150]
 [194 809]]


0.828